# Ejercicio práctico para Data Engineer Jr en Deacero.

## 1. Obtén los datos de las siguientes fuentes desde las apis

In [1]:
import pandas as pd
import numpy as np

In [2]:
URL_Pasajeros2016 = "http://analytics.deacero.com/Api/GetApi/ApiPasajeros2016/ecfb5fc7-0932-590f-832c-6d6055f2be07"
URL_Pasajeros2017 = "http://analytics.deacero.com/Api/GetApi/ApiPasajeros2017/faabd632-cc39-552d-a68b-02de4242f636"
URL_Viajes2016 = "http://analytics.deacero.com/Api/GetApi/ApiVuelos2016/9ea3b836-6938-52dc-9626-a8e35db81dd5"
URL_Viajes2017 = "http://analytics.deacero.com/Api/GetApi/ApiVuelos2017/fc126260-1cf8-5a46-995d-ba639ff5868b"
URL_Aerolineas = "http://analytics.deacero.com/Api/GetApi/ApiLineaAerea/1a8d9e13-ce30-50fc-bf34-6490eb799a75"

Parece que todos los archivos estan en formato Json.
Puedes ser leidos por pandas sin problema con la función read_json.

In [3]:
df_Pasajeros2016 = pd.read_json(URL_Pasajeros2016)
df_Pasajeros2017 = pd.read_json(URL_Pasajeros2017)
df_Viajes2016 = pd.read_json(URL_Viajes2016)
df_Viajes2017 = pd.read_json(URL_Viajes2017)
df_Aerolineas = pd.read_json(URL_Aerolineas)

## 2. Se tiene un requerimiento de análisis de las fuentes de datos de pasajeros y viajes. 

Como podrás observar, las listas se han estado llevando por año. 
En esta primera parte del reto se te pide:

* Unir cada conjunto de datos en una sola lista.
* Explicar el proceso realizado.
* En caso de detectar anomalías generadas por esta unión, deberás indicar el tipo de anomalía que se presenta, como se puede resolver y resolverlo de ser posible.


Primero que nada lo mejor sería ver un poco de contenido de cada set de datos leidos y obtener el esquema de los datos.

In [4]:
print(df_Pasajeros2016.info())
print(df_Pasajeros2017.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_Pasajero  100 non-null    int64 
 1   Pasajero     100 non-null    object
 2   Edad         100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID_Pasajero  100 non-null    int64 
 1   Pasajero     100 non-null    object
 2   Edad         100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB
None


In [5]:
df_Pasajeros2016.head()

,ID_Pasajero,Pasajero,Edad
0,576,Danielle Thompson,60
1,579,Natalie Cuevas,49
2,683,John Murray,28
3,681,Michael Jacobs,24
4,592,Brian Hunt,40


In [6]:
df_Pasajeros2017.head()

,ID_Pasajero,Pasajero,Edad
0,596,Javier Olson,71
1,625,Monique Ramirez,35
2,637,Rob Beeghly,29
3,730,Timothy Moore,21
4,682,Scot Wooten,72


### Unir conjunto de datos de pasajeros
Como tienen las mismas columnas y son del mismo tipos, basta con usar una concatenación de los datos usando pandas con la función concat.

In [7]:
df_Pasajeros = pd.concat([df_Pasajeros2016,df_Pasajeros2017])

###  Anomalías
Hasta este momento no se ha presentado ninguna anomalía. No se presentan campos nulos ni valores incorrectos en las columnas. Lo que podría ir mal es que haya registros duplicados o algo de ese estilo. Procedo a checar eso con un con el método value_counts() en la columna del ID_Pasajero. 

In [8]:
df_Pasajeros["ID_Pasajero"].value_counts()

682    12
38     10
582     6
693     6
637     5
       ..
650     1
652     1
653     1
655     1
632     1
Name: ID_Pasajero, Length: 133, dtype: int64

In [9]:
df_Pasajeros[df_Pasajeros["ID_Pasajero"]==682]

,ID_Pasajero,Pasajero,Edad
4,682,Scot Wooten,72
6,682,Scot Wooten,72
8,682,Scot Wooten,72
10,682,Scot Wooten,72
12,682,Scot Wooten,72
23,682,Scot Wooten,72
43,682,Scot Wooten,72
51,682,Scot Wooten,72
54,682,Scot Wooten,72
73,682,Scot Wooten,72


In [10]:
df_Pasajeros[df_Pasajeros["ID_Pasajero"]==38]

,ID_Pasajero,Pasajero,Edad
19,38,Anemone Ratner,39
21,38,Anemone Ratner,39
22,38,Anemone Ratner,39
25,38,Anemone Ratner,39
33,38,Anemone Ratner,39
40,38,Anemone Ratner,39
46,38,Anemone Ratner,39
61,38,Anemone Ratner,39
64,38,Anemone Ratner,39
80,38,Anemone Ratner,39


Parece que hay registros repetidos.

Hay que checar que los registros no se crucen entre ambas listas. Eso generaría problemas en la consistencia de los datos.
Se utilizaran conjuntos (set) y se realizara una intersección entre ellos con el método intersection.

In [11]:
set(df_Pasajeros2016["ID_Pasajero"]).intersection(set(df_Pasajeros2017["ID_Pasajero"]))

set()

Los registros no se traslapan. Solo habra que quitar los duplicados

In [12]:
df_Pasajeros.shape

(200, 3)

In [13]:
df_Pasajeros.drop_duplicates(inplace=True)

In [14]:
df_Pasajeros.shape

(138, 3)

Chequemos de nuevo que se hayan eliminado los registros duplicados.

In [15]:
df_Pasajeros["ID_Pasajero"].value_counts()

742    2
717    2
582    2
562    2
570    2
      ..
666    1
667    1
668    1
669    1
550    1
Name: ID_Pasajero, Length: 133, dtype: int64

In [16]:
df_Pasajeros[df_Pasajeros["ID_Pasajero"]==717]

,ID_Pasajero,Pasajero,Edad
11,717,Chad Wise,69
52,717,Billy Whitney,37


Se presenta otro problema. Los IDs que no son únicos para todos los pasajeros.
No parece existir una solucion sencilla a este problema. En este punto lo mejor es identificarlos para lidiar con ellos de la mejor forma mas adelante segun los requerimientos.

In [17]:
Lista_pasajeros_no_unicos = df_Pasajeros["ID_Pasajero"].value_counts()[df_Pasajeros["ID_Pasajero"].value_counts()>1].index

In [18]:
len(Lista_pasajeros_no_unicos)

5

### Unir conjunto de datos de viajes
De nuevo revisaremos los contenidos de los set de datos

In [19]:
print(df_Viajes2016.info())
print(df_Viajes2017.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Cve_LA       200 non-null    object
 1   Viaje        200 non-null    object
 2   Clase        200 non-null    object
 3   Precio       200 non-null    int64 
 4   Ruta         200 non-null    object
 5   Cve_Cliente  200 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 9.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Cve_LA       200 non-null    object
 1   Viaje        200 non-null    object
 2   Clase        200 non-null    object
 3   Precio       200 non-null    int64 
 4   Ruta         200 non-null    object
 5   Cve_Cliente  200 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 9.5+ KB
None


In [20]:
df_Viajes2016.head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,9/10/2016,Economy,60,DAL-MDW,553
1,AA,6/11/2016,Economy,150,DAL-SLC,554
2,DA,9/21/2016,Economy,68,DAL-AMA,554
3,UA,6/28/2016,Business,160,DAL-SLC,556
4,SW,6/25/2016,Economy,65,DAL-OKC,557


In [21]:
df_Viajes2017.head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,1/4/2017,Economy,86,DAL-ATL,637
1,SW,1/4/2017,First Class,237,DAL-ATL,402
2,SW,1/2/2017,Economy,62,DAL-LGA,191
3,SW,1/2/2017,Economy,81,DAL-MDW,191
4,AA,1/4/2017,Business,169,DAL-LGA,637


Como tienen las mismas columnas y son del mismo tipos, basta con usar una concatenación de los datos usando pandas con la función concat.

In [22]:
df_Viajes = pd.concat([df_Viajes2016,df_Viajes2017])

In [23]:
df_Viajes.head()

,Cve_LA,Viaje,Clase,Precio,Ruta,Cve_Cliente
0,SW,9/10/2016,Economy,60,DAL-MDW,553
1,AA,6/11/2016,Economy,150,DAL-SLC,554
2,DA,9/21/2016,Economy,68,DAL-AMA,554
3,UA,6/28/2016,Business,160,DAL-SLC,556
4,SW,6/25/2016,Economy,65,DAL-OKC,557


In [24]:
df_Viajes.shape

(400, 6)

### Anomalías
Sin un ID del viaje es mas complicado verificar que no existan registros duplicados. 
No parecen haber anomalias en este caso.

### Datos de aerolíneas

In [25]:
df_Aerolineas

,Code,Linea_Aerea
0,AA,American Airlines
1,SW,Southwest
2,AM,Aeromexico
3,AV,Avianca
4,KL,KLM


# 3. De lo obtenido anteriormente se requiere relacionar las listas de vuelos y pasajeros, que permitan analizar el perfil del pasajero por cada vuelo efectuado. 

De tal forma que se puedan obtener datos consolidados. En esta parte deberás:
* Explicar el proceso que utilizado para unir los pasajeros y los vuelos.
* Qué tipo de relación y por qué.

Se identifica la clave del pasajero en el conjunto de datos de Pasajeros como ID_Pasajero y en el de Vuelos como Cve_Cliente. Como la columna tiene nombres diferentes usaré la función merge. Se hace un inner join sobre los vuelos porque no se puede hacer un análisis de clientes y vuelos si falta información de los clientes.

In [26]:
dataset = pd.merge(df_Viajes,df_Pasajeros,how='inner',left_on="Cve_Cliente",right_on="ID_Pasajero")

In [27]:
dataset.shape

(434, 9)

Originalmente la tabla tenía 400 registros, después de hacer el cruce.
Los registros se duplican por los pasajeros duplicados. Lo mejor sera eliminarlos.

In [28]:
dataset.drop(dataset[dataset["ID_Pasajero"].isin(Lista_pasajeros_no_unicos)].index,axis=0,inplace=True)

# 4 Ahora se requiere que los datos consolidados de los vuelos y pasajeros se puedan unir con los datos de las Líneas Aéreas. 

En el caso de que la línea aérea no se pueda relacionar con la de vuelos y pasajeros se deberá indicar que se trata de “Otra” y finalmente se deberá dejar únicamente las columnas: 
* Fecha del viaje
* Clase
* Precio
* Ruta
* Edad
* Línea Aérea

En esta parte deberás indicar:

* ¿Qué tipo de proceso consideraste para unir los datos que se piden?
* ¿Qué columnas utilizaste para lograr esa relación?
* ¿Qué tipo de unión utilizaste para unir los datos?
* ¿Qué tipo de proceso utilizaste para dejar únicamente las columnas que se piden?

Se identifica la clave de la aerolínea en el conjunto de datos de Viajes como Cve_LA y en el de Aerolíneas como Code. 
Como la columna tiene nombres diferentes usaré la función merge. Se hace un left join sobre los viajes porque se deben de mantener los registros que tienen una línea aérea que no se encuentra en el conjunto de datos.

In [29]:
dataset = pd.merge(dataset,df_Aerolineas,how='left',left_on='Cve_LA',right_on='Code')

Para el caso donde no se pueda relacionar con la de vuelos y pasajeros se utiliza el método fillna() para llenar estos casos con "Otra".

In [30]:
dataset['Linea_Aerea'] = dataset['Linea_Aerea'].fillna("Otra")

La selección de variables la hago usando el método drop(). Pero primero realizo una copia de los datos por cualquier eventualidad es mejor tener el set original si se tiene la capacidad.

In [31]:
original_data = dataset.copy()

In [32]:
dataset.drop(["Cve_LA","Cve_Cliente","ID_Pasajero",'Pasajero','Code'],axis=1,inplace=True)

In [33]:
dataset.rename(columns={"Viaje":"Fecha"},inplace=True)

# 5. Por último, se requiere el promedio semestral (el primer semestre es de Ene - Jun y el segundo es de Jul - Dic) del precio agrupado por Año, Clase, Ruta y las Línea Aérea como columnas.

Primero se le dará formato de fecha a la columna de Fecha para poder utilizar las funciones de pandas especializadas.

In [34]:
dataset['Fecha'] = pd.to_datetime(dataset['Fecha'])

Antes que nada, creo una copia de los datos donde agregaré columnas útiles para la agrupación

In [35]:
data_grouped = dataset.copy()

Se crea la variable del semestre a partir del mes de la fecha del viaje. De la misma forma se agrega el año.

In [36]:
data_grouped["Semestre"]="Enero-Junio"
data_grouped.loc[pd.DatetimeIndex(data_grouped['Fecha']).month > 6,"Semestre"]='Julio-Diciembre'
data_grouped["Anio"]=pd.DatetimeIndex(data_grouped['Fecha']).year

Por último, se crea la agregación utilizando tablas pivote, usando el metodo pivot_table de pandas.

In [37]:
pd.set_option("display.max.rows",None)

Aqui hay una duda si lo que se necesita es tener el semestre como un nivel de agrupación para comparar entre los dos semestres o lo que se requiere es promediar los precios de los vuelos del semestre y luego volver a promediarlos. Pienso que es la segunda pero de todas formas agrego debajo el otro caso.

In [71]:
paso1_data_grouped = data_grouped.pivot_table(values='Precio',columns = 'Semestre',
                                              index=["Anio","Clase","Ruta","Linea_Aerea"])

In [72]:
paso1_data_grouped.head()

Semestre                                 Enero-Junio  Julio-Diciembre
Anio Clase    Ruta    Linea_Aerea                                    
2016 Business DAL-AMA American Airlines         58.0            186.0
                      Avianca                  132.0              NaN
                      KLM                      184.0              NaN
              DAL-ATL Aeromexico               218.0              NaN
                      American Airlines        102.0            135.0

In [73]:
paso2_data_grouped = np.mean(paso1_data_grouped,axis=1)

In [74]:
paso2_data_grouped

Anio  Clase        Ruta     Linea_Aerea      
2016  Business     DAL-AMA  American Airlines    122.000000
                            Avianca              132.000000
                            KLM                  184.000000
                   DAL-ATL  Aeromexico           218.000000
                            American Airlines    118.500000
                            Avianca              107.000000
                            KLM                  227.000000
                            Southwest            185.000000
                   DAL-AUS  Aeromexico           250.000000
                            American Airlines    186.500000
                            KLM                  224.500000
                   DAL-ELP  Aeromexico           176.000000
                            American Airlines    114.000000
                            Avianca              192.500000
                            KLM                  157.500000
                   DAL-GRR  Aeromexico           269.0

Interpretación agrupando por semestre.

In [41]:
data_grouped.pivot_table(values='Precio',index=["Anio","Semestre","Clase","Ruta","Linea_Aerea"])

Precio
Anio Semestre        Clase       Ruta    Linea_Aerea                  
2016 Enero-Junio     Business    DAL-AMA American Airlines   58.000000
                                         Avianca            132.000000
                                         KLM                184.000000
                                 DAL-ATL Aeromexico         218.000000
                                         American Airlines  102.000000
                                 DAL-AUS Aeromexico         250.000000
                                         American Airlines  186.500000
                                         KLM                258.000000
                                 DAL-ELP Avianca            173.000000
                                         KLM                153.000000
                                 DAL-GRR Aeromexico         269.000000
                                         American Airlines  221.000000
                                         Avianca            149.000000
                                         Southwest          266.000000
                                 DAL-HOU Aeromexico          76.000000
                                         American Airlines  121.000000
                                         Avianca            172.000000
                                         KLM                 62.000000
                                         Southwest           71.000000
                                 DAL-LGA American Airlines  215.000000
                                         Avianca            191.000000
                                         Otra               146.000000
                                         Southwest           97.000000
                                 DAL-OKC American Airlines  181.000000
                                         Southwest           89.000000
                                 DAL-PHX Avianca            236.000000
                                         Southwest          202.000000
                                 DAL-SLC Avianca            117.000000
                                         KLM                 75.000000
                                         Otra               160.000000
                                         Southwest          224.000000
                                 DAL-TUL KLM                182.000000
                                         Southwest          265.000000
                     Economy     DAL-AMA Aeromexico         178.000000
                                         American Airlines  130.500000
                                         KLM                 62.000000
                                         Southwest           66.000000
                                 DAL-AUS Aeromexico          97.000000
                                         American Airlines  133.000000
                                         KLM                192.000000
                                         Otra                60.000000
                                         Southwest           95.000000
                                 DAL-ELP Aeromexico         245.000000
                                         American Airlines  191.000000
                                 DAL-GRR Aeromexico         253.000000
                                         American Airlines   75.500000
                                         Avianca             71.000000
                                         KLM                177.000000
                                 DAL-HOU American Airlines   71.000000
                                         KLM                 52.000000
                                 DAL-LGA Aeromexico         207.000000
                                         American Airlines  140.000000
                                         Avianca            238.000000
                                 DAL-MDW American Airlines  199.000000
                                         Avianca             79.000000
                 